In [48]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import time
import random

#function to scrape website
def web_scrape_1year(url, season, team_name):
    
    #custom user agent to fake identity so site doesn't block scraper
    request_headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
    }
    
    #source
    source = requests.get(url, headers=request_headers).text
    soup = BeautifulSoup(source, 'html.parser')
    
    #league of the team
    league = soup.find('div', class_='clubs')
    league1 = league.find('span', class_='header_end')
    finalleague = league1.text.replace('(', '').replace(')', '')
    
    if finalleague != 'Premier League':
        print('Not Premier League Season')
        return None

    #table
    table = soup.find('table', class_='stats_table sortable min_width')

    #scraping the titles of columns
    instance = []
    for header in table.find_all('thead'):
        for row in header.find_all('th'):
            if 'over_header' in row.get('class', []):
                continue
            if '' in row:
                continue
            instance.append(row.text.strip())
    instance = list(filter(None, instance))
        
    #list stats of every team of first table
    modest = []
    arsenal = soup.find('tbody')
    for arsenal2 in arsenal.find_all('tr'):
        for lister in arsenal2.find_all(['th', 'td']):
            modest.append(lister.text.strip())


    # function that changes the modest list from being flat to a 2d table
    def flat_to_df(flatlist, num_columns, column_names=None):
        table_rows = [flatlist[i:i + num_columns] for i in range(0, len(flatlist), num_columns)]
        
        df = pd.DataFrame(table_rows, columns=column_names)
        return df
    
    #if statement to handle if data is incomplete
    if len(modest) % len(instance) != 0:
        print(f'DATA MISMATCH for {team_name} in {season}')
        return None
    
    #using flat to df function
    num_cols = len(instance)
    col_names = instance
    df_data = flat_to_df(modest, num_cols, col_names)
    df_data.insert(0, 'Season', season)
    df_data.insert(3, 'Team', team_name)
    return df_data

#team dictionary name+urlcode for looping through urls for teams
teams = {
    'Arsenal': '18bb7c10',
    'Aston-Villa': '8602292d',
    'Bournemouth': '4ba7cbea',
    'Brentford': 'cd051869',
    'Brighton-and-Hove-Albion': 'd07537b9',
    'Cardiff-City': '75fae011',
    'Chelsea': 'cff3d9bb',
    'Crystal-Palace': '47c64c55',
    'Everton': 'd3fd31cc',
    'Fulham': 'fd962109',
    'Huddersfield-Town': 'f5922ca5',
    'Ipswich-Town': 'b74092de',
    'Leeds-United': '5bfb9659',
    'Leicester-City': 'a2d435b3',
    'Liverpool': '822bd0ba',
    'Manchester-City': 'b8fd03ef',
    'Manchester-United': '19538871',
    'Newcastle-United': 'b2b47a98',
    'Norwich-City': '1c781004',
    'Nottingham-Forest': 'e4a775cb',
    'Southampton': '33c895d4',
    'Sheffield-United': '1df6b87e',
    'Stoke-City': '17892952',
    'Swansea-City': 'fb10988f',
    'Tottenham-Hotspur': '361ca564',
    'Watford': '2abfe087',
    'West-Bromwich-Albion': '60c6b05f',
    'West-Ham-United': '7c21e445',
    'Wolverhampton-Wanderers': '8cec06e1'
}

#most recent season swish to go through
year1 = 2023
year2 = 2024

#setting to compare to outputted columns, if wrong can continue loop and skip
expected_columns = 36

#period length: how many years back want to go
perlength = 6

confin = []
for i in range(perlength):
    for team_name, team_code in teams.items():
        url = f'https://fbref.com/en/squads/{team_code}/{year1}-{year2}/{team_name}-Stats'
        print(f"--- Scraping: {team_name} for season {year1}-{year2} ---")
        con = web_scrape_1year(url, f'{year1}-{year2}', f'{team_name}')
        sleep_duration = random.uniform(5, 15)
        time.sleep(sleep_duration)
        if con is None:
            print('Skipping, None!')
            continue
        if expected_columns != con.shape[1]:
            print('Skipping, columns dont align')
            continue
        else:
            confin.append(con)
    sleep_duration = random.uniform(20, 40)
    time.sleep(sleep_duration)
    year1 -= 1
    year2 -= 1
    

result = pd.concat(confin, ignore_index=True)
print(result)
    
result.to_csv('PremierFull2018-2024.csv', encoding='utf-8-sig', index=False)


--- Scraping: Arsenal for season 2023-2024 ---
--- Scraping: Aston-Villa for season 2023-2024 ---
--- Scraping: Bournemouth for season 2023-2024 ---
--- Scraping: Brentford for season 2023-2024 ---
--- Scraping: Brighton-and-Hove-Albion for season 2023-2024 ---
--- Scraping: Cardiff-City for season 2023-2024 ---
Not Premier League Season
Skipping, None!
--- Scraping: Chelsea for season 2023-2024 ---
--- Scraping: Crystal-Palace for season 2023-2024 ---
--- Scraping: Everton for season 2023-2024 ---
--- Scraping: Fulham for season 2023-2024 ---
--- Scraping: Huddersfield-Town for season 2023-2024 ---
Not Premier League Season
Skipping, None!
--- Scraping: Ipswich-Town for season 2023-2024 ---
Not Premier League Season
Skipping, None!
--- Scraping: Leeds-United for season 2023-2024 ---
Not Premier League Season
Skipping, None!
--- Scraping: Leicester-City for season 2023-2024 ---
Not Premier League Season
Skipping, None!
--- Scraping: Liverpool for season 2023-2024 ---
--- Scraping: Manc

In [49]:
df = pd.read_csv('PremierFull2018-2024.csv')

In [50]:
df

,Season,Player,Nation,Team,Pos,Age,MP,Starts,Min,90s,...,Ast.1,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Matches
0,2023-2024,William Saliba,fr FRA,Arsenal,DF,22,38,38,"3,420",38.0,...,0.03,0.08,0.05,0.08,0.04,0.01,0.05,0.04,0.05,Matches
1,2023-2024,Declan Rice,eng ENG,Arsenal,MF,24,38,37,"3,225",35.8,...,0.22,0.42,0.20,0.42,0.09,0.15,0.24,0.09,0.24,Matches
2,2023-2024,Martin Ødegaard,no NOR,Arsenal,MF,24,35,35,"3,091",34.3,...,0.29,0.52,0.17,0.47,0.22,0.28,0.50,0.17,0.45,Matches
3,2023-2024,Ben White,eng ENG,Arsenal,DF,25,37,35,"2,988",33.2,...,0.12,0.24,0.12,0.24,0.03,0.11,0.15,0.03,0.15,Matches
4,2023-2024,Bukayo Saka,eng ENG,Arsenal,FW,21,35,35,"2,919",32.4,...,0.28,0.77,0.31,0.59,0.48,0.32,0.80,0.33,0.65,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3809,2018-2019,Will Norris,eng ENG,Wolverhampton-Wanderers,GK,24,1,0,1,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
3810,2018-2019,Bright Enobakhare,ng NGA,Wolverhampton-Wanderers,"FW,MF",20,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches
3811,2018-2019,Ryan Giles,eng ENG,Wolverhampton-Wanderers,"DF,MF",18,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches
3812,2018-2019,Kortney Hause,eng ENG,Wolverhampton-Wanderers,DF,23,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches


In [68]:
key_df = pd.read_csv('/Users/ogizelenovic/Downloads/FOOTBALL/Key.csv', index_col='Column Name')

In [70]:
key_df = key_df.drop('Column1', axis=1)

In [81]:
key_df.sort_index(inplace=True)

In [82]:
key_df

,Column meaning
Column Name,
90s,Minutes played divided by 90
Age,Age at season start
Ast,Assists per 90 minutes
Ast,Assists
Column,Meaning
CrdR,Red Cards
CrdY,Yellow Card
G+A,Goals and Assists per 90 minutes
G+A,Goals + Assists


In [51]:
filt = (df['Gls'] > 30)

In [52]:
df.loc[filt, ['Player', 'Season', 'Gls', 'Team']]

,Player,Season,Gls,Team
1115,Erling Haaland,2022-2023,36.0,Manchester-City
